In [1]:
from torch import nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import time
from torchsummary import summary
import config
from facenet_pytorch import training
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from PIL import Image
import glob
import torchvision.models as models
from util import AverageMeter, learning_rate_decay, Logger
import math

In [2]:
from models.alexnet import alexnet
alexnet = alexnet()['backbone']
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=40, bias=True)
alexnet = alexnet.cuda()
summary(alexnet,(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 31, 31]          23,296
              ReLU-2           [-1, 64, 31, 31]               0
         MaxPool2d-3           [-1, 64, 15, 15]               0
            Conv2d-4          [-1, 192, 15, 15]         307,392
              ReLU-5          [-1, 192, 15, 15]               0
         MaxPool2d-6            [-1, 192, 7, 7]               0
            Conv2d-7            [-1, 384, 7, 7]         663,936
              ReLU-8            [-1, 384, 7, 7]               0
            Conv2d-9            [-1, 256, 7, 7]         884,992
             ReLU-10            [-1, 256, 7, 7]               0
           Conv2d-11            [-1, 256, 7, 7]         590,080
             ReLU-12            [-1, 256, 7, 7]               0
        MaxPool2d-13            [-1, 256, 3, 3]               0
AdaptiveAvgPool2d-14            [-1, 25

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [3]:
# Root directory for dataset
data_root = "/home/mehmetyavuz/datasets/CelebA128/"
attr_root = "/home/mehmetyavuz/datasets/list_attr_celeba.txt"
# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 64

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = (128,128)
epochs = 100

In [4]:
class CelebA(data.Dataset):
    def __init__(self, data_path, attr_path, image_size, mode, selected_attrs):
        super(CelebA, self).__init__()
        self.data_path = data_path
        att_list = open(attr_path, 'r', encoding='utf-8').readlines()[1].split()
        atts = [att_list.index(att) + 1 for att in selected_attrs]
        images = np.loadtxt(attr_path, skiprows=2, usecols=[0], dtype=np.str)
        labels = np.loadtxt(attr_path, skiprows=2, usecols=atts, dtype=np.int)
        
        self.tf = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
        if mode == 'train':
            self.images = images[:162770]
            self.labels = labels[:162770]

        if mode == 'valid':
            self.images = images[162770:182637]
            self.labels = labels[162770:182637]

        if mode == 'test':
            self.images = images[182637:]
            self.labels = labels[182637:]
                                       
        self.length = len(self.images)
    def __getitem__(self, index):
        img = self.tf(Image.open(os.path.join(self.data_path, self.images[index])))
        att = torch.tensor((self.labels[index] + 1) // 2)
        return img, att.to(torch.float32)
    def __len__(self):
        return self.length

In [5]:
attrs_default = ["5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs", "Big_Lips", "Big_Nose", 
                 "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows", "Chubby", "Double_Chin", "Eyeglasses", "Goatee", 
                 "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", 
                 "Oval_Face", "Pale_Skin", "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair", 
                 "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace", "Wearing_Necktie", "Young"]

In [6]:
dataset = CelebA(data_root, attr_root, image_size, 'test', attrs_default)
test_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=workers)

In [7]:
# Decide which device we want to run on
device = torch.device("cuda:0")

In [8]:
alexnet = torch.nn.DataParallel(alexnet)

In [16]:
weights = torch.load("alexnet_wYFCC_CelebA.pth")

In [17]:
alexnet.load_state_dict(weights,strict=True)

<All keys matched successfully>

In [18]:
features = torch.FloatTensor(len(dataset), 40)
targets = torch.LongTensor(len(dataset),40)

In [19]:
ptr = 0
for i, batch in enumerate(test_loader):
    images = batch[0].cuda(non_blocking=True)
    target = batch[1].cuda(non_blocking=True)
    output = alexnet(images)
    output = nn.Sigmoid()(output)
    
    b = output.size(0)
    
    features[ptr:ptr+b].copy_(output.detach())
    targets[ptr:ptr+b].copy_(target.detach())
    ptr += b

In [20]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [21]:
for i, att in enumerate(attrs_default):
    print(att, ';', (targets[:,i] == torch.round(features[:,i])).float().mean().item())
print('Average Accuracy;', (targets == torch.round(features)).float().mean().item())

5_o_Clock_Shadow ; 0.9333733916282654
Arched_Eyebrows ; 0.8213104605674744
Attractive ; 0.8119426965713501
Bags_Under_Eyes ; 0.8345355987548828
Bald ; 0.9869251847267151
Bangs ; 0.952660083770752
Big_Lips ; 0.7006812691688538
Big_Nose ; 0.8349363803863525
Black_Hair ; 0.882376492023468
Blond_Hair ; 0.95421302318573
Blurry ; 0.9563170075416565
Brown_Hair ; 0.868800699710846
Bushy_Eyebrows ; 0.9201482534408569
Chubby ; 0.9526099562644958
Double_Chin ; 0.9607253670692444
Eyeglasses ; 0.9910830855369568
Goatee ; 0.9691413640975952
Gray_Hair ; 0.9801121950149536
Heavy_Makeup ; 0.905520498752594
High_Cheekbones ; 0.861687183380127
Male ; 0.9754533767700195
Mouth_Slightly_Open ; 0.9209998846054077
Mustache ; 0.9664863348007202
Narrow_Eyes ; 0.8651938438415527
No_Beard ; 0.9473499655723572
Oval_Face ; 0.7440637350082397
Pale_Skin ; 0.9630798697471619
Pointy_Nose ; 0.7600440979003906
Receding_Hairline ; 0.9324216246604919
Rosy_Cheeks ; 0.9460976123809814
Sideburns ; 0.9729986786842346
Smiling ;

In [22]:
for i, att in enumerate(attrs_default):
    print(att, ';', roc_auc_score(targets.cpu().numpy()[:,i], features.cpu().numpy()[:,i]))
print('Average ROC-AUC;', roc_auc_score(targets.cpu().numpy(), features.cpu().numpy()))

5_o_Clock_Shadow ; 0.9524515498856321
Arched_Eyebrows ; 0.886710470948659
Attractive ; 0.9022451902891208
Bags_Under_Eyes ; 0.8653141648817456
Bald ; 0.9878856580347216
Bangs ; 0.9801308123331682
Big_Lips ; 0.7147545281744155
Big_Nose ; 0.8505320841299961
Black_Hair ; 0.9483282358235869
Blond_Hair ; 0.9771232714197933
Blurry ; 0.8925097902360941
Brown_Hair ; 0.9137597558613909
Bushy_Eyebrows ; 0.9242122549668921
Chubby ; 0.9332736912882683
Double_Chin ; 0.939676525697232
Eyeglasses ; 0.9921539905881706
Goatee ; 0.9767191368145696
Gray_Hair ; 0.9752906437510129
Heavy_Makeup ; 0.9690874105275645
High_Cheekbones ; 0.937046822413569
Male ; 0.996351608268794
Mouth_Slightly_Open ; 0.9737654137092846
Mustache ; 0.9630940391557563
Narrow_Eyes ; 0.816190873772795
No_Beard ; 0.9773798861681661
Oval_Face ; 0.7357570346677796
Pale_Skin ; 0.9261406706809907
Pointy_Nose ; 0.7736466739865241
Receding_Hairline ; 0.9285025989795382
Rosy_Cheeks ; 0.9464499663839703
Sideburns ; 0.9761899642691328
Smiling